In [71]:
import collections
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder


In [72]:
!pip install scikit-surprise

You should consider upgrading via the '/Users/antonioalmeidab/Code/ml-school/.env/bin/python3.9 -m pip install --upgrade pip' command.


In [73]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [74]:
reviews_tv_df = pd.read_csv('reviews_tv_shows.csv')
reviews_tv_df =  reviews_tv_df.drop(['id_y', 'Unnamed: 0'], axis=1)

labelencoder = LabelEncoder()
reviews_tv_df['tv_show'] = labelencoder.fit_transform(reviews_tv_df['tv_show'])
reviews_tv_df['user'] = labelencoder.fit_transform(reviews_tv_df['user'])
reviews_tv_df

,tv_show,16,18,35,37,80,878,9648,10751,10759,10762,10763,10764,10765,10766,10767,10768,user,rating
0,86,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,15,10.000000
1,173,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,78,5.195675
2,173,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,10,5.195675
3,176,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,112,10.000000
4,180,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,62,5.195675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7562,5,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,132,10.000000
7563,131,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,166,10.000000
7564,74,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,92,8.000000
7565,181,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,68,4.000000


In [75]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(reviews_tv_df[['user', 'tv_show', 'rating']], reader)



In [76]:
param_grid = {'k': [20, 50, 100, 200],
              'min_k': [1, 2, 5, 9, 17]}
gridsearch_KNN = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5)
gridsearch_KNN.fit(data)

print('Best Score: ' + str(gridsearch_KNN.best_score['rmse']))
print('Best Params: ' + str(gridsearch_KNN.best_params['rmse']))


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [77]:
knn_model = KNNBasic(**gridsearch_KNN.best_params['rmse'])
knn_model.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


In [78]:
param_grid = {'n_epochs': [20, 25, 30, 50, 60],
              'lr_all': [0.002, 0.005],
              'reg_all': [0.04, 0.06]}

gridsearch_SVD = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)

gridsearch_SVD.fit(data)

print("Best Score:" + str(gridsearch_SVD.best_score['rmse']))
print("Best Params:" + str(gridsearch_SVD.best_params['rmse']))


Best Score:3.204792496692817
Best Params:{'n_epochs': 20, 'lr_all': 0.002, 'reg_all': 0.04}


In [79]:
svd_model = SVD(**gridsearch_SVD.best_params['rmse'])
svd_model.fit(data.build_full_trainset())

print(svd_model.predict(uid=20, iid=100).est)
print(svd_model.predict(uid=25, iid=80).est)
print(svd_model.predict(uid=15, iid=86).est)

5.926930402523384
5.200250676102917
5.34654630208286


In [80]:
print(svd_model.predict(uid=86, iid=15).est)


6.601898601373762


In [81]:
results_SVD = cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
results_KNN = cross_validate(knn_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.2431  3.1511  3.1816  3.1648  3.2175  3.1916  0.0340  
MAE (testset)     2.8020  2.7193  2.7193  2.7223  2.7815  2.7489  0.0356  
Fit time          0.68    0.59    0.47    0.47    0.62    0.57    0.08    
Test time         0.13    0.01    0.02    0.01    0.06    0.05    0.04    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.2013  3.1668  3.1504  3.1877  3.2236  3.1859  0.0256

In [82]:
compare_models = {
    'SVD': {'RMSE': results_SVD['test_rmse'][-1],'MAE': results_SVD['test_mae'][-1]},
    'KNNBasic': {'RMSE': results_KNN['test_rmse'][-1],'MAE': results_KNN['test_mae'][-1]}
}
compare_models

{'SVD': {'RMSE': 3.217470415543223, 'MAE': 2.7814919319693963},
 'KNNBasic': {'RMSE': 3.2235863182650353, 'MAE': 2.756514404380967}}